<a href="https://colab.research.google.com/github/VinishUchiha/NewsSummarizationUsingTransformers/blob/master/News_Article_Summarization_Using_Transformers(T5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/huggingface/transformers
!pip install ./transformers

Cloning into 'transformers'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 25889 (delta 132), reused 218 (delta 104), pack-reused 25615
Receiving objects: 100% (25889/25889), 16.83 MiB | 16.18 MiB/s, done.
Resolving deltas: 100% (18089/18089), done.
Processing ./transformers
     |████████████████████████████████| 3.8MB 6.1MB/s 
     |████████████████████████████████| 1.0MB 37.4MB/s 
     |████████████████████████████████| 890kB 51.2MB/s 
  Created wheel for transformers: filename=transformers-2.9.0-cp36-none-any.whl size=629384 sha256=2e6e57ab9da07cfbb69fb7f9f36835534ee0623385567f7912b0a2a0bbc27df3
  Stored in directory: /tmp/pip-ephem-wheel-cache-2xcd6e64/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9d130c67a7c415c72ef513564d8e0c81278338a8e06ff220c7de2aaff26579

In [0]:
import requests
from bs4 import BeautifulSoup
import pprint
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [0]:
url = "https://www.rediff.com/news"
r = requests.get(url)
page = r.content

In [0]:
soup = BeautifulSoup(page, 'html5lib')

In [0]:
coverpage_news = soup.find_all('div', class_='newleftcontainer')

In [0]:
news = coverpage_news[0].select('h2 a')

In [0]:
texts = []
links = []
for tag in news:
  texts.append(tag.text)
  links.append(tag.attrs['href'])

In [0]:
#remove the first link because its a just a news updates
texts = texts[1:]
links = links[1:]

In [0]:
#first 5 article 
num_of_article = 5

news_contents = []
for i in range(0,num_of_article):
      # Reading the content (it is divided in paragraphs)
    article = requests.get(links[i])
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='arti_contentbig')
    x = body[0].find_all('p')    
    # Unifying the paragraphs
    list_paragraphs = []
    for p in range(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

In [13]:
news_contents

['A Mig-29 multi-role fighter aircraft of the Indian Air Force crashed in Chuharpur village near Jalandhar in Punjab on Friday after it developed a technical snag, officials said. \xa0  The pilot of the jet ejected safely, they said. "One Mig-29 aircraft airborne on a training mission from an Air Force base near Jalandhar met with an accident. The aircraft had developed a technical snag and the pilot ejected safely as he was unable to control the aircraft," the IAF said in a statement. An IAF fighter jet crashed into the fields here, Nawanshahr Deputy Commissioner Vinay Bublani said. Officials said the crash took place around 10.30 am. The IAF said the pilot has been rescued by a helicopter and that a court of inquiry has been ordered to investigate the cause of the accident. At present, the IAF has three squadrons of the Soviet-era MiG-29 fighter jets and two of them are stationed at the Adampur Air Force station. One squadron comprises 16-18 aircraft. The fleet was upgraded around tw

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained("t5-large")
model.to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-large")

In [0]:
summary = []
def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  for i in range(0, len(lst), n):
    yield lst[i : i + n]


def generate_summaries(lns, batch_size, device):
  pp = pprint.PrettyPrinter(indent=0, width=120)

  # update config with summarization specific params
  task_specific_params = model.config.task_specific_params
  if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))

  for batch in list(chunks(lns, batch_size)):
    batch = [model.config.prefix + text for text in batch]

    dct = tokenizer.batch_encode_plus(batch, max_length=512, return_tensors="pt", pad_to_max_length=True)
    input_ids = dct["input_ids"].to(device)
    attention_mask = dct["attention_mask"].to(device)

    summaries = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    dec = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]

    for hypothesis in dec:
      s = pp.pformat(hypothesis)
      s = s.replace("'", "").replace("(", "").replace(")", "")
      summary.append(hypothesis)
      print(s, end='\n')
      print("=" * 120)

In [29]:
generate_summaries(news_contents, batch_size=5, device=device)

the pilot of the jet ejected safely, officials say . an IAF fighter jet crashed into the fields here . the IAF has 
 lost a number of mig 29s in crashes in the last few years .
government urges people to make COVID-19 prevention guidelines a part of their lives . health ministry says peak of 
 "disease can be avoided if prescribed dos and donts are followed . peak of the disease in india may be reached in "
 June or July, according to modelling data .
class 10 and 12 exams were postponed due to the COVID-19-induced lockdown . the schedule has been decided in order 
 to ensure that the board exams are completed before competitive examinations . class 10 exams are only pending in 
 north east Delhi where they were affected due to law and order situation .
brihanmumbai municipal corporation commissioner Pravin pardeshi transferred . he was replaced as Mumbai commissioner 
 by I SChahal, currently additional chief secretary urban development Pardeshi has been posted in the urban 
 "develop

In [30]:
summary

['the pilot of the jet ejected safely, officials say . an IAF fighter jet crashed into the fields here . the IAF has lost a number of mig 29s in crashes in the last few years .',
 "government urges people to make COVID-19 prevention guidelines a part of their lives . health ministry says peak of disease can be avoided if prescribed dos and don'ts are followed . peak of the disease in india may be reached in June or July, according to modelling data .",
 'class 10 and 12 exams were postponed due to the COVID-19-induced lockdown . the schedule has been decided in order to ensure that the board exams are completed before competitive examinations . class 10 exams are only pending in north east Delhi where they were affected due to law and order situation .',
 "brihanmumbai municipal corporation commissioner Pravin pardeshi transferred . he was replaced as Mumbai commissioner by I SChahal, currently additional chief secretary (urban development) Pardeshi has been posted in the urban develop

In [31]:
import pandas as pd
df = pd.DataFrame({'title':texts[:5],'url':links[:5],'content':news_contents,'summary':summary})
df.head()

,title,url,content,summary
0,"IAF's MiG-29 crashes in Punjab, pilot safe",https://www.rediff.com/news/report/iafs-mig-29...,A Mig-29 multi-role fighter aircraft of the In...,"the pilot of the jet ejected safely, officials..."
1,"Learn to live with virus, follow guidelines: G...",https://www.rediff.com/news/report/learn-to-li...,"""We have to learn to live with the virus,"" the...",government urges people to make COVID-19 preve...
2,"Class 10, 12 CBSE exams to be held from July 1-15",https://www.rediff.com/news/report/class-10-12...,"Pending Class 10 and 12 CBSE exams, which were...",class 10 and 12 exams were postponed due to th...
3,Mumbai civic chief shunted out amid rising Cov...,https://www.rediff.com/news/report/mumbai-civi...,Amid the growing number of coronavirus cases i...,brihanmumbai municipal corporation commissione...
4,"Finish Babri trial against Advani, Joshi by Au...",https://www.rediff.com/news/report/finish-babr...,The Supreme Court on Friday extended by three ...,the top court asked special judge to control t...


In [0]:
#export the csv
df.to_csv('news_summaries.csv')

In [17]:
!nvidia-smi

Fri May  8 16:41:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    34W /  70W |   4865MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  